In [141]:
import pandas as pd
# from dowhy import gcm
import numpy as np
import matplotlib.pyplot as plt
import gc

import os

In [208]:
node = "sauber"

In [209]:
#import exp_filtered
data = pd.read_csv('../Datasets/experiments_valid.csv',header=0, index_col=0)
experiments = pd.DataFrame(data)

experiments.tail(10)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
409,891,2024-02-15 19:25:21.9807959,2024-02-15 19:55:21.9832958,validator,random33,19,8,23,4,8,2,0.25,3.00,1708025121,1708026921
412,898,2024-02-15 23:20:24.1114255,2024-02-15 23:50:24.1747705,validator,random5,8,6,16,4,8,2,0.25,0.25,1708039224,1708041024
413,899,2024-02-15 23:53:58.9555104,2024-02-16 00:23:59.0175630,validator,random5,8,6,16,4,8,2,0.25,0.25,1708041238,1708043039
415,901,2024-02-16 01:01:07.9360686,2024-02-16 01:31:07.9411590,validator,random6,16,8,20,6,8,2,0.25,1.00,1708045267,1708047067
416,902,2024-02-16 01:34:42.7924620,2024-02-16 02:04:42.7974999,validator,random6,16,8,20,6,8,2,0.25,1.00,1708047282,1708049082
418,904,2024-02-16 02:41:52.3057095,2024-02-16 03:11:52.3106811,validator,random7,10,8,16,4,8,4,0.25,1.00,1708051312,1708053112
419,905,2024-02-16 03:15:26.8680015,2024-02-16 03:45:26.8705969,validator,random7,10,8,16,4,8,4,0.25,1.00,1708053326,1708055126
421,907,2024-02-16 04:22:35.4565262,2024-02-16 04:52:35.4619071,validator,random8,10,8,16,4,8,2,0.15,1.00,1708057355,1708059155
422,908,2024-02-16 04:56:10.5331406,2024-02-16 05:26:10.5389618,validator,random8,10,8,16,4,8,2,0.15,1.00,1708059370,1708061170
423,909,2024-02-16 05:29:45.3956405,2024-02-16 05:59:45.3984223,validator,reference,8,6,12,4,8,2,0.25,1.00,1708061385,1708063185


In [210]:
messages = pd.DataFrame()

for index, row in experiments.iterrows():
    data = pd.read_csv('../Datasets/partialTables/mess_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_mess = pd.DataFrame(data)
    query_mess = query_mess.drop(columns=["start", "end"])

    messages = pd.concat([query_mess, messages])
    del query_mess
    gc.collect()

    if os.path.exists('../Datasets/partialTables/ihave_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/ihave_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_ihave = pd.DataFrame(data)
        query_ihave["type"] = '17.0'
        query_ihave = query_ihave.drop(columns=["_measurement"])
        
        messages = pd.concat([query_ihave, messages])
        del query_ihave
        gc.collect()
    
    if os.path.exists('../Datasets/partialTables/iwant_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/iwant_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_iwant = pd.DataFrame(data)
        query_iwant["type"] = '16.0'
        query_iwant = query_iwant.drop(columns=["_measurement"])
    
        messages = pd.concat([query_iwant, messages])
        del query_iwant
        gc.collect()
    

messages.head(10)

,_time,experiment,type
0,2024-02-16 05:36:59.009457+00:00,909,16.0
0,2024-02-16 05:36:59.009392+00:00,909,17.0
1,2024-02-16 05:36:59.017823+00:00,909,17.0
2,2024-02-16 05:36:59.173290+00:00,909,17.0
3,2024-02-16 05:36:59.173299+00:00,909,17.0
4,2024-02-16 05:37:00.009414+00:00,909,17.0
5,2024-02-16 05:37:00.017322+00:00,909,17.0
6,2024-02-16 05:37:00.173046+00:00,909,17.0
7,2024-02-16 05:37:00.173063+00:00,909,17.0
8,2024-02-16 05:37:00.173067+00:00,909,17.0


In [211]:
# Transpose the table -> type and count
df = messages#.drop(columns=['identifier'])
df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],df['type'],pd.Grouper(key="_time", freq='5s')])['index'].count().reset_index()


pivoted = pd.pivot_table(by_time,index=["experiment", pd.Grouper(freq="5s", key="_time")], columns="type", values="index", margins=False).fillna(0)
pivoted = pivoted.reset_index().rename_axis(None, axis=1)

pivoted.head(100)

/tmp/ipykernel_82388/2738252698.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,2.0,3.0,11.0,12.0,16.0,17.0
0,390,2023-10-04 18:52:00+00:00,605.0,69.0,0.0,0.0,0.0,0.0
1,390,2023-10-04 18:52:05+00:00,837.0,95.0,0.0,0.0,0.0,70.0
2,390,2023-10-04 18:52:10+00:00,1374.0,158.0,0.0,0.0,0.0,69.0
3,390,2023-10-04 18:52:15+00:00,1237.0,142.0,0.0,0.0,0.0,36.0
4,390,2023-10-04 18:52:20+00:00,946.0,111.0,0.0,0.0,0.0,71.0
...,...,...,...,...,...,...,...,...
95,391,2023-10-04 19:43:05+00:00,1382.0,184.0,0.0,0.0,0.0,38.0
96,391,2023-10-04 19:43:10+00:00,1102.0,150.0,0.0,0.0,0.0,64.0
97,391,2023-10-04 19:43:15+00:00,1465.0,194.0,0.0,0.0,0.0,58.0
98,391,2023-10-04 19:43:20+00:00,1392.0,184.0,0.0,0.0,0.0,38.0


In [212]:
#Rename
eventsTS = pivoted.rename(columns={2.0:'messageDuplicated', 3.0: 'messageReceived', 11.0: 'graft', 12.0: 'prune', '16.0': 'iwant', '17.0':'ihave'})

# eventsTS.to_csv('./events20s.csv')
# eventsTS.head(500)

In [213]:
#Import partial tables and append -> for propagation time

publishMessage = pd.DataFrame()
deliverMessage = pd.DataFrame()

query_mess = pd.DataFrame()
query_pub = pd.DataFrame()

for index, row in experiments.iterrows():
    if os.path.exists('../Datasets/partialTables/mess2_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/mess2_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_mess = pd.DataFrame(data)
        deliverMessage = pd.concat([query_mess, deliverMessage])
        del query_mess

    if os.path.exists('../Datasets/partialTables/pub_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../Datasets/partialTables/pub_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_pub = pd.DataFrame(data)
        publishMessage = pd.concat([publishMessage, query_pub])
        del query_pub

    gc.collect()

deliverMessage.head(10)


,_time,messageID,experiment
0,2024-02-16 05:36:59.010244+00:00,ACQIARIg3KUU2uoJavOyzcVNRJLjO/l6ByIRKxWBd/gS56...,909
1,2024-02-16 05:37:00.144274+00:00,ACQIARIgk0rNydMyK7+0tmIuH0VRAI1V/5Qz7Zpu3tZczx...,909
2,2024-02-16 05:37:00.736606+00:00,ACQIARIg3KUU2uoJavOyzcVNRJLjO/l6ByIRKxWBd/gS56...,909
3,2024-02-16 05:37:00.857081+00:00,ACQIARIgfl7Wkeq304/jtKRO9+BFSqGblHME8UBHqKYV1B...,909
4,2024-02-16 05:37:01.005360+00:00,ACQIARIgDyYdTccrMyzaXMarwZL3ceYmaR0Y/e7nmYKRbb...,909
5,2024-02-16 05:37:01.018524+00:00,ACQIARIg2E5YMFznHacBOytrj+HcCNQBtpgMqFDEHtnRlQ...,909
6,2024-02-16 05:37:03.147477+00:00,ACQIARIgk0rNydMyK7+0tmIuH0VRAI1V/5Qz7Zpu3tZczx...,909
7,2024-02-16 05:37:03.741137+00:00,ACQIARIg3KUU2uoJavOyzcVNRJLjO/l6ByIRKxWBd/gS56...,909
8,2024-02-16 05:37:03.863901+00:00,ACQIARIgfl7Wkeq304/jtKRO9+BFSqGblHME8UBHqKYV1B...,909
9,2024-02-16 05:37:04.012520+00:00,ACQIARIgDyYdTccrMyzaXMarwZL3ceYmaR0Y/e7nmYKRbb...,909


In [214]:
publishMessage['_time'] = pd.to_datetime(publishMessage["_time"], format='mixed')#.tz_localize(None)
deliverMessage['_time'] = pd.to_datetime(deliverMessage["_time"], format='mixed')#.tz_localize(None)

# def calcAverageTime(publish, received, expTime, parameter):
expTime = experiments
publish = publishMessage
received = deliverMessage

publish = publish[['_time', 'messageID','experiment']]
received = received[['_time', 'messageID', 'experiment']]

joined = publish.merge(received, on=['messageID', 'experiment'])
joined['diff'] = ((joined['_time_y'] - joined['_time_x'])/ pd.Timedelta(microseconds=1)).astype(int)
joined = joined.loc[joined["diff"] >= 0].dropna()
joined.head(10)


,_time_x,messageID,experiment,_time_y,diff
0,2023-10-04 19:06:30.415374+00:00,ACQIARIgyvYjy7hyNGKdR2St/W+uynGD3jRp3L9MblBO1t...,390,2023-10-04 19:06:30.415605+00:00,231
1,2023-10-04 19:06:30.415701+00:00,ACQIARIgyvYjy7hyNGKdR2St/W+uynGD3jRp3L9MblBO1t...,390,2023-10-04 19:06:30.416214+00:00,513
2,2023-10-04 19:06:30.415998+00:00,ACQIARIgyvYjy7hyNGKdR2St/W+uynGD3jRp3L9MblBO1t...,390,2023-10-04 19:06:30.416513+00:00,515
3,2023-10-04 19:06:30.416459+00:00,ACQIARIgyvYjy7hyNGKdR2St/W+uynGD3jRp3L9MblBO1t...,390,2023-10-04 19:06:30.416783+00:00,324
4,2023-10-04 19:06:30.416761+00:00,ACQIARIgyvYjy7hyNGKdR2St/W+uynGD3jRp3L9MblBO1t...,390,2023-10-04 19:06:30.417195+00:00,434
5,2023-10-04 19:06:30.417031+00:00,ACQIARIgyvYjy7hyNGKdR2St/W+uynGD3jRp3L9MblBO1t...,390,2023-10-04 19:06:30.418134+00:00,1103
6,2023-10-04 19:06:30.417523+00:00,ACQIARIgyvYjy7hyNGKdR2St/W+uynGD3jRp3L9MblBO1t...,390,2023-10-04 19:06:30.418209+00:00,686
7,2023-10-04 19:06:30.417915+00:00,ACQIARIgyvYjy7hyNGKdR2St/W+uynGD3jRp3L9MblBO1t...,390,2023-10-04 19:06:30.418692+00:00,777
8,2023-10-04 19:06:33.420697+00:00,ACQIARIgyvYjy7hyNGKdR2St/W+uynGD3jRp3L9MblBO1t...,390,2023-10-04 19:06:33.421047+00:00,350
9,2023-10-04 19:06:33.421137+00:00,ACQIARIgyvYjy7hyNGKdR2St/W+uynGD3jRp3L9MblBO1t...,390,2023-10-04 19:06:33.422169+00:00,1032


In [215]:
df = joined.drop(columns=['_time_x', 'messageID']).rename(columns={'_time_y' : '_time'})

df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],pd.Grouper(key="_time", freq='5s')])['diff'].agg('mean').reset_index()
by_time.head(100)

/tmp/ipykernel_82388/2407133802.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,diff
0,390,2023-10-04 19:06:30+00:00,646.312500
1,390,2023-10-04 19:06:35+00:00,798.812500
2,390,2023-10-04 19:06:40+00:00,367.750000
3,390,2023-10-04 19:06:45+00:00,561.750000
4,390,2023-10-04 19:06:50+00:00,320.062500
...,...,...,...
95,391,2023-10-04 19:46:10+00:00,510.125000
96,391,2023-10-04 19:46:15+00:00,564.791667
97,391,2023-10-04 19:46:20+00:00,409.625000
98,391,2023-10-04 19:46:25+00:00,325.750000


In [216]:
prop = by_time.rename(columns={'diff':'propagationTime'})

prop = prop.merge(eventsTS, on=['experiment', '_time'], how='outer')
prop.head(100)

,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave
0,390,2023-10-04 19:06:30+00:00,646.312500,1521.0,194.0,0.0,0.0,0.0,69.0
1,390,2023-10-04 19:06:35+00:00,798.812500,1543.0,191.0,0.0,0.0,0.0,49.0
2,390,2023-10-04 19:06:40+00:00,367.750000,1191.0,142.0,0.0,0.0,0.0,80.0
3,390,2023-10-04 19:06:45+00:00,561.750000,1691.0,208.0,0.0,0.0,0.0,69.0
4,390,2023-10-04 19:06:50+00:00,320.062500,1527.0,191.0,0.0,0.0,0.0,50.0
...,...,...,...,...,...,...,...,...,...
95,391,2023-10-04 19:46:10+00:00,510.125000,1314.0,167.0,0.0,0.0,0.0,64.0
96,391,2023-10-04 19:46:15+00:00,564.791667,1061.0,151.0,0.0,0.0,0.0,58.0
97,391,2023-10-04 19:46:20+00:00,409.625000,1490.0,194.0,0.0,0.0,0.0,38.0
98,391,2023-10-04 19:46:25+00:00,325.750000,1284.0,162.0,0.0,0.0,0.0,64.0


In [217]:
rpc = pd.DataFrame()

for idx, exec in experiments.iterrows():
    data = pd.read_csv('../Datasets/partialTables/rpc_'+node+'_'+str(exec['experiment'])+'.csv', header=0,  index_col=0)
    query_rpc = pd.DataFrame(data)
        
    rpc = pd.concat([rpc,query_rpc])

    del query_rpc
    gc.collect()
        

rpc.head(10)

,_time,_measurement,experiment,start,end
0,2023-10-04 18:52:04.136190+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
1,2023-10-04 18:52:04.136297+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
2,2023-10-04 18:52:04.136337+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
3,2023-10-04 18:52:04.136532+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
4,2023-10-04 18:52:04.136756+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
5,2023-10-04 18:52:04.136763+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
6,2023-10-04 18:52:04.137046+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
7,2023-10-04 18:52:04.137073+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
8,2023-10-04 18:52:04.137283+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
9,2023-10-04 18:52:04.137298+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518


In [218]:
rpc = rpc.drop(columns=['start', 'end'])
                       
rpc.reset_index(inplace=True)
rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)
by_time = rpc.groupby([rpc['experiment'],pd.Grouper(key="_time", freq='5s')])['index'].agg('count').reset_index()
# by_time.head(100)

rpc=by_time.rename(columns={'index':'rpc'})

/tmp/ipykernel_82388/1580870231.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)


In [219]:
message=received

message.reset_index(inplace=True)
message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)
by_time = message.groupby([message['experiment'],pd.Grouper(key="_time", freq='5s')])['messageID'].agg('count').reset_index()
message = by_time

message.head(10)

/tmp/ipykernel_82388/1432395461.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,messageID
0,390,2023-10-04 18:52:00+00:00,69
1,390,2023-10-04 18:52:05+00:00,95
2,390,2023-10-04 18:52:10+00:00,158
3,390,2023-10-04 18:52:15+00:00,142
4,390,2023-10-04 18:52:20+00:00,111
5,390,2023-10-04 18:52:25+00:00,158
6,390,2023-10-04 18:52:30+00:00,135
7,390,2023-10-04 18:52:35+00:00,144
8,390,2023-10-04 18:52:40+00:00,162
9,390,2023-10-04 18:52:45+00:00,121


In [220]:
message = message.rename(columns={'messageID':'count'})
rpc = rpc.rename(columns={'rpc': 'count'})
total_bandwidth = pd.concat([rpc, message])

total_bandwidth.reset_index(inplace=True)
total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)
by_time = total_bandwidth.groupby([total_bandwidth['experiment'],pd.Grouper(key="_time", freq='5s')])['count'].agg('sum').reset_index()
by_time.head(100)

/tmp/ipykernel_82388/448925867.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,count
0,390,2023-10-04 18:52:00+00:00,743
1,390,2023-10-04 18:52:05+00:00,1062
2,390,2023-10-04 18:52:10+00:00,1724
3,390,2023-10-04 18:52:15+00:00,1539
4,390,2023-10-04 18:52:20+00:00,1204
...,...,...,...
95,391,2023-10-04 19:43:05+00:00,1757
96,391,2023-10-04 19:43:10+00:00,1419
97,391,2023-10-04 19:43:15+00:00,1864
98,391,2023-10-04 19:43:20+00:00,1767


In [221]:
by_time['totalBandwidth'] = by_time['count']

totalBandwidth = by_time.drop(columns=['count'])
totalBandwidth.head(10)

,experiment,_time,totalBandwidth
0,390,2023-10-04 18:52:00+00:00,743
1,390,2023-10-04 18:52:05+00:00,1062
2,390,2023-10-04 18:52:10+00:00,1724
3,390,2023-10-04 18:52:15+00:00,1539
4,390,2023-10-04 18:52:20+00:00,1204
5,390,2023-10-04 18:52:25+00:00,1731
6,390,2023-10-04 18:52:30+00:00,1471
7,390,2023-10-04 18:52:35+00:00,1577
8,390,2023-10-04 18:52:40+00:00,1765
9,390,2023-10-04 18:52:45+00:00,1315


In [222]:
#gossip bandwidth
gossipBandwidth = rpc
gossipBandwidth['gossipReceived'] = gossipBandwidth['count']
gossipBandwidth['gossipBandwidth'] = gossipBandwidth['count']/5
gossipBandwidth = gossipBandwidth.drop(columns=['count'])

gossipBandwidth.head(100)

,experiment,_time,gossipReceived,gossipBandwidth
0,390,2023-10-04 18:52:00+00:00,674,134.8
1,390,2023-10-04 18:52:05+00:00,967,193.4
2,390,2023-10-04 18:52:10+00:00,1566,313.2
3,390,2023-10-04 18:52:15+00:00,1397,279.4
4,390,2023-10-04 18:52:20+00:00,1093,218.6
...,...,...,...,...
95,391,2023-10-04 19:43:05+00:00,1573,314.6
96,391,2023-10-04 19:43:10+00:00,1269,253.8
97,391,2023-10-04 19:43:15+00:00,1670,334.0
98,391,2023-10-04 19:43:20+00:00,1583,316.6


In [223]:
#MessageBandwidth

messageBandwidth = message
messageBandwidth['messageBandwidth'] = messageBandwidth['count']/5
messageBandwidth = messageBandwidth.drop(columns=['count'])

messageBandwidth.head(10)

,experiment,_time,messageBandwidth
0,390,2023-10-04 18:52:00+00:00,13.8
1,390,2023-10-04 18:52:05+00:00,19.0
2,390,2023-10-04 18:52:10+00:00,31.6
3,390,2023-10-04 18:52:15+00:00,28.4
4,390,2023-10-04 18:52:20+00:00,22.2
5,390,2023-10-04 18:52:25+00:00,31.6
6,390,2023-10-04 18:52:30+00:00,27.0
7,390,2023-10-04 18:52:35+00:00,28.8
8,390,2023-10-04 18:52:40+00:00,32.4
9,390,2023-10-04 18:52:45+00:00,24.2


In [224]:
prop['_time'] = pd.to_datetime(prop["_time"], format='mixed', infer_datetime_format=True)
messageBandwidth['_time'] = pd.to_datetime(messageBandwidth["_time"], format='mixed', infer_datetime_format=True)

messBand = prop.merge(messageBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

messBand.head(500)

/tmp/ipykernel_82388/2873079763.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  prop['_time'] = pd.to_datetime(prop["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_82388/2873079763.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  messageBandwidth['_time'] = pd.to_datetime(messageBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth
0,390,2023-10-04 19:06:30+00:00,646.3125,1521.0,194.0,0.0,0.0,0.0,69.0,38.8
1,390,2023-10-04 19:06:35+00:00,798.8125,1543.0,191.0,0.0,0.0,0.0,49.0,38.2
2,390,2023-10-04 19:06:40+00:00,367.7500,1191.0,142.0,0.0,0.0,0.0,80.0,28.4
3,390,2023-10-04 19:06:45+00:00,561.7500,1691.0,208.0,0.0,0.0,0.0,69.0,41.6
4,390,2023-10-04 19:06:50+00:00,320.0625,1527.0,191.0,0.0,0.0,0.0,50.0,38.2
...,...,...,...,...,...,...,...,...,...,...
495,402,2023-10-05 02:04:10+00:00,307.3750,309.0,89.0,0.0,0.0,0.0,62.0,17.8
496,402,2023-10-05 02:04:15+00:00,425.0625,347.0,105.0,0.0,0.0,0.0,63.0,21.0
497,402,2023-10-05 02:04:20+00:00,227.4375,435.0,126.0,0.0,0.0,0.0,63.0,25.2
498,402,2023-10-05 02:04:25+00:00,247.2500,328.0,89.0,0.0,0.0,0.0,61.0,17.8


In [225]:
messBand['_time'] = pd.to_datetime(messBand["_time"], format='mixed', infer_datetime_format=True)
totalBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

totalBand = messBand.merge(totalBandwidth, on=['experiment', '_time'], how='outer').fillna(0)
totalBand['totalBandwidth'] = (totalBand['totalBandwidth']+totalBand['messageDuplicated'])/5

totalBand.head(500)

/tmp/ipykernel_82388/511213079.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  messBand['_time'] = pd.to_datetime(messBand["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_82388/511213079.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  totalBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth
0,390,2023-10-04 19:06:30+00:00,646.3125,1521.0,194.0,0.0,0.0,0.0,69.0,38.8,688.8
1,390,2023-10-04 19:06:35+00:00,798.8125,1543.0,191.0,0.0,0.0,0.0,49.0,38.2,696.6
2,390,2023-10-04 19:06:40+00:00,367.7500,1191.0,142.0,0.0,0.0,0.0,80.0,28.4,539.8
3,390,2023-10-04 19:06:45+00:00,561.7500,1691.0,208.0,0.0,0.0,0.0,69.0,41.6,762.6
4,390,2023-10-04 19:06:50+00:00,320.0625,1527.0,191.0,0.0,0.0,0.0,50.0,38.2,690.2
...,...,...,...,...,...,...,...,...,...,...,...
495,402,2023-10-05 02:04:10+00:00,307.3750,309.0,89.0,0.0,0.0,0.0,62.0,17.8,164.6
496,402,2023-10-05 02:04:15+00:00,425.0625,347.0,105.0,0.0,0.0,0.0,63.0,21.0,184.8
497,402,2023-10-05 02:04:20+00:00,227.4375,435.0,126.0,0.0,0.0,0.0,63.0,25.2,227.8
498,402,2023-10-05 02:04:25+00:00,247.2500,328.0,89.0,0.0,0.0,0.0,61.0,17.8,171.8


In [226]:
totalBand['_time'] = pd.to_datetime(totalBand["_time"], format='mixed', infer_datetime_format=True)
gossipBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

gossipBand = totalBand.merge(gossipBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

gossipBand.head(500)

/tmp/ipykernel_82388/187803194.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  totalBand['_time'] = pd.to_datetime(totalBand["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_82388/187803194.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  gossipBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipReceived,gossipBandwidth
0,390,2023-10-04 19:06:30+00:00,646.3125,1521.0,194.0,0.0,0.0,0.0,69.0,38.8,688.8,1729.0,345.8
1,390,2023-10-04 19:06:35+00:00,798.8125,1543.0,191.0,0.0,0.0,0.0,49.0,38.2,696.6,1749.0,349.8
2,390,2023-10-04 19:06:40+00:00,367.7500,1191.0,142.0,0.0,0.0,0.0,80.0,28.4,539.8,1366.0,273.2
3,390,2023-10-04 19:06:45+00:00,561.7500,1691.0,208.0,0.0,0.0,0.0,69.0,41.6,762.6,1914.0,382.8
4,390,2023-10-04 19:06:50+00:00,320.0625,1527.0,191.0,0.0,0.0,0.0,50.0,38.2,690.2,1733.0,346.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,402,2023-10-05 02:04:10+00:00,307.3750,309.0,89.0,0.0,0.0,0.0,62.0,17.8,164.6,425.0,85.0
496,402,2023-10-05 02:04:15+00:00,425.0625,347.0,105.0,0.0,0.0,0.0,63.0,21.0,184.8,472.0,94.4
497,402,2023-10-05 02:04:20+00:00,227.4375,435.0,126.0,0.0,0.0,0.0,63.0,25.2,227.8,578.0,115.6
498,402,2023-10-05 02:04:25+00:00,247.2500,328.0,89.0,0.0,0.0,0.0,61.0,17.8,171.8,442.0,88.4


In [227]:
consDiscovery = gossipBand.rename(columns={'messageDuplicated' : 'messageOverhead'})#.drop(columns=['bandwidth'])
consDiscovery.head(10)

,experiment,_time,propagationTime,messageOverhead,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipReceived,gossipBandwidth
0,390,2023-10-04 19:06:30+00:00,646.3125,1521.0,194.0,0.0,0.0,0.0,69.0,38.8,688.8,1729.0,345.8
1,390,2023-10-04 19:06:35+00:00,798.8125,1543.0,191.0,0.0,0.0,0.0,49.0,38.2,696.6,1749.0,349.8
2,390,2023-10-04 19:06:40+00:00,367.7500,1191.0,142.0,0.0,0.0,0.0,80.0,28.4,539.8,1366.0,273.2
3,390,2023-10-04 19:06:45+00:00,561.7500,1691.0,208.0,0.0,0.0,0.0,69.0,41.6,762.6,1914.0,382.8
4,390,2023-10-04 19:06:50+00:00,320.0625,1527.0,191.0,0.0,0.0,0.0,50.0,38.2,690.2,1733.0,346.6
5,390,2023-10-04 19:06:55+00:00,769.8750,1142.0,138.0,0.0,0.0,0.0,80.0,27.6,518.4,1312.0,262.4
6,390,2023-10-04 19:07:00+00:00,381.0000,1674.0,208.0,0.0,0.0,0.0,70.0,41.6,755.6,1896.0,379.2
7,390,2023-10-04 19:07:05+00:00,672.4375,1436.0,180.0,0.0,0.0,0.0,50.0,36.0,649.4,1631.0,326.2
8,390,2023-10-04 19:07:10+00:00,421.7500,1069.0,132.0,0.0,0.0,0.0,80.0,26.4,486.8,1233.0,246.6
9,390,2023-10-04 19:07:15+00:00,603.6875,1680.0,208.0,0.0,0.0,0.0,69.0,41.6,758.2,1903.0,380.6


In [228]:
consFinal = consDiscovery.merge(experiments, on='experiment')
consFinal = consFinal.drop(columns=['_time', 'start', 'end', 'startUnix', 'endUnix', 'parameter'])#, 'originalStart', 'originalEnd']) 

consFinal.to_csv('../Datasets/1node/raw/consolidated5s_'+node+'.csv')
consFinal.head(10)


,experiment,propagationTime,messageOverhead,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,...,gossipBandwidth,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval
0,390,646.3125,1521.0,194.0,0.0,0.0,0.0,69.0,38.8,688.8,...,345.8,unl,8,6,12,4,8,2,0.25,1.0
1,390,798.8125,1543.0,191.0,0.0,0.0,0.0,49.0,38.2,696.6,...,349.8,unl,8,6,12,4,8,2,0.25,1.0
2,390,367.7500,1191.0,142.0,0.0,0.0,0.0,80.0,28.4,539.8,...,273.2,unl,8,6,12,4,8,2,0.25,1.0
3,390,561.7500,1691.0,208.0,0.0,0.0,0.0,69.0,41.6,762.6,...,382.8,unl,8,6,12,4,8,2,0.25,1.0
4,390,320.0625,1527.0,191.0,0.0,0.0,0.0,50.0,38.2,690.2,...,346.6,unl,8,6,12,4,8,2,0.25,1.0
5,390,769.8750,1142.0,138.0,0.0,0.0,0.0,80.0,27.6,518.4,...,262.4,unl,8,6,12,4,8,2,0.25,1.0
6,390,381.0000,1674.0,208.0,0.0,0.0,0.0,70.0,41.6,755.6,...,379.2,unl,8,6,12,4,8,2,0.25,1.0
7,390,672.4375,1436.0,180.0,0.0,0.0,0.0,50.0,36.0,649.4,...,326.2,unl,8,6,12,4,8,2,0.25,1.0
8,390,421.7500,1069.0,132.0,0.0,0.0,0.0,80.0,26.4,486.8,...,246.6,unl,8,6,12,4,8,2,0.25,1.0
9,390,603.6875,1680.0,208.0,0.0,0.0,0.0,69.0,41.6,758.2,...,380.6,unl,8,6,12,4,8,2,0.25,1.0
